<a href="https://colab.research.google.com/github/sitikanthaprusty2904/Pytorch_sp/blob/main/ANN_fmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

#ok bro

In [94]:
# set random seed for reproductibility
torch.manual_seed(42)


In [95]:
!wget https://raw.githubusercontent.com/google/colabtools/master/notebooks/samples/fmnist_small.csv

--2025-11-01 10:52:00--  https://raw.githubusercontent.com/google/colabtools/master/notebooks/samples/fmnist_small.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-11-01 10:52:00 ERROR 404: Not Found.



In [96]:
df=pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [97]:
# train test split
X=df.iloc[:,1:]
y=df.iloc[:,0]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [98]:
# scaling as max pixel is 255
X_train=X_train/255.0
X_test=X_test/255.0

In [99]:
class Customdataset(Dataset):
  def __init__(self,x,y):
    self.x=torch.tensor(x.values,dtype=torch.float32)
    self.y=torch.tensor(y.values,dtype=torch.long)
  def __len__(self):
    return len(self.x)
  def __getitem__(self, index) :
    return self.x[index],self.y[index]


In [100]:
# create train dataset object
train_dataset=Customdataset(X_train,y_train)
train_dataloader=DataLoader(train_dataset,batch_size=480,shuffle=True)

#create test data set object
test_dataset=Customdataset(X_test,y_test)
test_dataloader=DataLoader(test_dataset,batch_size=480,shuffle=False)

In [101]:
# nn module class
class MyNN(nn.Module):
  def __init__(self,x_num):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(x_num,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)

        )
  def forward(self,x):
    return self.model(x)


In [107]:
# set lr and epoch
lr=0.01
epoch=100

In [108]:
# initiating the model
model=MyNN(X_train.shape[1])

#loss
loss=nn.CrossEntropyLoss()

#optimizer
optimizer=optim.Adam(model.parameters(),lr=lr)

# len(train_dataloader)

In [109]:
# training loop

for epoch in range (epoch):
  tl=0
  for batch_featurs,batch_labels in train_dataloader:
    # forward pass
    output=model(batch_featurs)
    #loss
    loss_value=loss(output,batch_labels)
    #backward
    optimizer.zero_grad()
    loss_value.backward()
    #update grad
    optimizer.step()
    tl+=loss_value.item()
  print(f"epoch {epoch+1} loss {tl/len(train_dataloader)}")


epoch 1 loss 1.5533648550510406
epoch 2 loss 0.8460027396678924
epoch 3 loss 0.6801770985126495
epoch 4 loss 0.59178626537323
epoch 5 loss 0.5228501498699188
epoch 6 loss 0.47497796416282656
epoch 7 loss 0.4265164941549301
epoch 8 loss 0.40024428963661196
epoch 9 loss 0.37373778223991394
epoch 10 loss 0.3752553254365921
epoch 11 loss 0.33319239020347596
epoch 12 loss 0.3202696621417999
epoch 13 loss 0.305499404668808
epoch 14 loss 0.26715346425771713
epoch 15 loss 0.28573338240385054
epoch 16 loss 0.25944431126117706
epoch 17 loss 0.2375685006380081
epoch 18 loss 0.22508087009191513
epoch 19 loss 0.21983290016651152
epoch 20 loss 0.23417114168405534
epoch 21 loss 0.20745291858911513
epoch 22 loss 0.19861726462841034
epoch 23 loss 0.19989935606718062
epoch 24 loss 0.17794054597616196
epoch 25 loss 0.16828498989343643
epoch 26 loss 0.17230023592710494
epoch 27 loss 0.19841287583112716
epoch 28 loss 0.17972193360328675
epoch 29 loss 0.1698627382516861
epoch 30 loss 0.1516713321208954
epoc

In [110]:
# set model for evaluation
model.eval()  # mode change for evaluation


MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [111]:
# evaluation code
total=0
correct=0

with torch.no_grad():
  for batch_featurs,batch_labels in test_dataloader:
    output=model(batch_featurs)
    _,predicted=torch.max(output.data,1)
    total+= batch_labels.shape[0]
    correct+= (predicted==batch_labels).sum().item()
print(f"Accuracy: {correct/total}")


Accuracy: 0.8441666666666666
